# Import and preparation

In [1]:
import sys
import logging

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## Logger Preparation

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

c_handler = logging.StreamHandler(sys.stdout)
# c_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_format = logging.Formatter(f'%(levelname)-8s: %(message)s')
c_handler.setFormatter(c_format)

for handler in logger.handlers:
    logger.removeHandler(handler)
logger.addHandler(c_handler)

In [3]:
logger.info('test')
logger.debug('aku adalah anak gembala')

INFO    : test


## Data Reading

In [47]:
SQUAD_DATASET_PATH = 'Datasets/SQuAD/v2.0/dev-v2.0-translated.json'

df_squad = pd.read_json(SQUAD_DATASET_PATH)
df_squad = df_squad
print(df_squad.shape)
print(df_squad)

df_squad_original = pd.read_json('Datasets/SQuAD/v2.0/dev-v2.0.json')
print(df_squad_original.shape)
df_squad_original.head()

(35, 2)
                                            title  \
0                                       Normandia   
1                    Teori kompleksitas komputasi   
2                              California Selatan   
3                          Sky (Kerajaan Inggris)   
4                            Victoria (Australia)   
5                                        Huguenot   
6                                       Mesin uap   
7                                         Oksigen   
8                              Krisis minyak 1973   
9                                 Hukum Uni Eropa   
10                             hutan hujan Amazon   
11                                     Ctenophora   
12                             Fresno, California   
13                               Pergantian paket   
14                                 Kematian kelam   
15                                        Geologi   
16                                        Farmasi   
17                            Pembangk

,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."


In [48]:
df_squad_original['title'] = df_squad_original['data'].apply(lambda x: x.get('title'))
df_squad_original['paragraphs'] = df_squad_original['data'].apply(lambda x: x.get('paragraphs'))
df_squad_original.drop(columns=['data'], inplace=True)

print(df_squad_original.iloc[-1]['paragraphs'][0]['context'])
df_squad_original.iloc[-1]['paragraphs'][0]['qas']

Philosophers in antiquity used the concept of force in the study of stationary and moving objects and simple machines, but thinkers such as Aristotle and Archimedes retained fundamental errors in understanding force. In part this was due to an incomplete understanding of the sometimes non-obvious force of friction, and a consequently inadequate view of the nature of natural motion. A fundamental error was the belief that a force is required to maintain motion, even at a constant velocity. Most of the previous misunderstandings about motion and force were eventually corrected by Galileo Galilei and Sir Isaac Newton. With his mathematical insight, Sir Isaac Newton formulated laws of motion that were not improved-on for nearly three hundred years. By the early 20th century, Einstein developed a theory of relativity that correctly predicted the action of forces on objects with increasing momenta near the speed of light, and also provided insight into the forces produced by gravitation and 

[{'question': 'What concept did philosophers in antiquity use to study simple machines?',
  'id': '573735e8c3c5551400e51e71',
  'answers': [{'text': 'force', 'answer_start': 46},
   {'text': 'force', 'answer_start': 46},
   {'text': 'the concept of force', 'answer_start': 31},
   {'text': 'the concept of force', 'answer_start': 31},
   {'text': 'force', 'answer_start': 46},
   {'text': 'force', 'answer_start': 46}],
  'is_impossible': False},
 {'question': 'What was the belief that maintaining motion required force?',
  'id': '573735e8c3c5551400e51e72',
  'answers': [{'text': 'fundamental error', 'answer_start': 387},
   {'text': 'A fundamental error', 'answer_start': 385},
   {'text': 'A fundamental error', 'answer_start': 385},
   {'text': 'A fundamental error', 'answer_start': 385},
   {'text': 'A fundamental error', 'answer_start': 385},
   {'text': 'A fundamental error', 'answer_start': 385}],
  'is_impossible': False},
 {'question': 'Who had mathmatical insite?',
  'id': '573735e

# Data preprocessing

In [188]:
# # for topic_idx in df_squad.iloc[0]['paragraphs']:
# for question in df_squad.iloc[1]['paragraphs'][6]['qas']:
#     answers = question['answers'][0]
#     print(answers['text'])
# #     # 0,0
# #     if answers['text'] == 'Berbahaya dalam Cinta':
# #         answers['text'] = 'Dangerously in Love'
# #     elif answers['text'] == 'bernyanyi dan menari':
# #         answers['text'] = 'menyanyi dan menari'
# #     # 0,1
# #     if answers['text'] == 'Beyonce':
# #         answers['text'] = 'Beyoncé '
# #     # 0,3
# #     if answers['text'] == 'Metodis':
# #         answers['text'] = 'Metodhis'
# #     elif answers['text'] == 'Amerika Afrika':
# #         answers['text'] = 'Afrika-Amerika'
# #     # 1,0
#     if answers['text'] == 'hanya perlahan':
#         answers['text'] = 'piano solo'
# #     # 1,3
#     if answers['text'] == 'rencana':
#         answers['text'] = 'piano'
#     print(answers['text'])
#     print()

Oktober
Oktober

Perancis
Perancis

alasan Istana
alasan Istana

seruling dan biola
seruling dan biola

penyakit
penyakit

Oktober 1810
Oktober 1810

Lyceum Warsawa
Lyceum Warsawa

seruling dan biola
seruling dan biola

rencana
piano

enam bulan
enam bulan

Perancis
Perancis

seruling dan biola
seruling dan biola

rencana
piano

Istana Saxon.
Istana Saxon.



In [7]:
import string
from fuzzywuzzy import fuzz

FIND_STRING_APPROX_ATTEMPT_LIMIT = 3
SHORT_ANSWER_THRESHOLD = 2
SENTENCE_SIMILARITY_THRESHOLD = 0.5
WORD_SIMILARITY_THRESHOLD = 80
NOT_FOUND_VALUE = (-1, -1)

def remove_punctuations(text):
    table = str.maketrans('', '', string.punctuation)
    filtered = [w.translate(table) for w in text.split()]
    return ' '.join(filtered)

def find_word_in_list_approx(list_word, word, start_search_idx=0):
    found = False
    i = start_search_idx
    while i < len(list_word) and not found:
        found = fuzz.token_set_ratio(list_word[i], word) > WORD_SIMILARITY_THRESHOLD
        i += 1
    if found:
        return i-1
    else:
        return -1

def find_string_approx(context, answer, logger=logger):
    context_words = context.split()
    answer_words = answer.split()
    context_words_length = len(context_words)
    answer_words_length = len(answer_words)
    
    # string.find only used for numeric, else use fuzzywuzzy
    if answer.strip().isnumeric():
        char_pos_start = context.find(answer)
        char_pos_end = char_pos_start + len(answer)
        return (char_pos_start, char_pos_end)
        
    found = False        
    attempt = 0
    
    i_initial = -1
    j_initial = 0    
    i_start = i_initial
    j_start = j_initial
    while attempt < FIND_STRING_APPROX_ATTEMPT_LIMIT and not found:
        if attempt > 0: logger.debug(f'Trying another starting word. Attempt {attempt+1}..')
        while (i_start == i_initial or i_start == -1) and j_start < answer_words_length:
            i_start = find_word_in_list_approx(context_words, answer_words[j_start], start_search_idx=i_initial+1)
#             logger.debug(answer_words[j_start])
#             logger.debug(i_start)
            if i_start == -1:
                i_initial = -1
                j_start += 1
        if i_start == -1:
            return NOT_FOUND_VALUE

#         logger.debug(answer_words[j_start])
#         logger.debug(i_start)
        match_count = 0
        i = i_start
        j = j_start
        while i < context_words_length and j < answer_words_length:
#             logger.debug(f'{context_words[i]} vs {answer_words[j]}')
            if fuzz.token_set_ratio(context_words[i], answer_words[j]) > WORD_SIMILARITY_THRESHOLD:
                match_count += 1
                i += 1
                j += 1
            else:
                if j < answer_words_length-1 and fuzz.token_set_ratio(context_words[i], answer_words[j+1]) > WORD_SIMILARITY_THRESHOLD:
                    match_count += 1
                    i += 1
                    j += 2
                elif i < context_words_length-1 and fuzz.token_set_ratio(context_words[i+1], answer_words[j]) > WORD_SIMILARITY_THRESHOLD:
                    match_count += 1
                    i += 2
                    j += 1
                else:
                    i += 1
                    j += 1
                    
        similarity = match_count / answer_words_length
        if similarity >= SENTENCE_SIMILARITY_THRESHOLD and not (answer_words_length == 2 and similarity == 0.5): # half correct for 2 length words is considered unsimilar            
            char_pos_start = 0
            start_idx = i_start
            for k in range(start_idx):
                char_pos_start += len(context_words[k]) + 1
                
            char_pos_end = 0
            end_idx = i
            for k in range(start_idx, end_idx):
                char_pos_end += len(context_words[k]) + 1
            found = True
        else:
            attempt += 1
            logger.debug('Not found..')
            i_initial = i_start    
            
    if found:
        logger.debug(f'start:({i_start}, {j_start}), end:({i-1}, {j-1}), ({context_words[i_start]},{answer_words[j_start]}), {match_count}/{answer_words_length}={similarity:.2f}')
        return (char_pos_start, char_pos_start+char_pos_end-1)        
    else:
        logger.debug(f'({i-1},{j-1}): {match_count}/{answer_words_length}={similarity:.2f}')
        return NOT_FOUND_VALUE

In [8]:
find_string_approx('at, republik meskipun terdapat berbagai kepentingan dalam konflik atau dengan para pejuang, seperti Denmark-Norwegia. Republik Belanda, sekutu lama Inggris, mempertahankan kenetralannya, takut akan kemungkinan melawan Inggris dan Prusia melawan kekuatan besar Eropa, bahkan beru', 'Republik Belanda')
find_string_approx('Buddhisme / ˈbudɪzəm / adalah agama nonthisti [catatan 1] atau filsafat (bahasa Sansekerta: धर्म dharma; Pali: धम्धम dhamma) yang mencakup berbagai tradisi, kepercayaan, dan praktik spiritual yang sebagian besar didasarkan pada ajaran yang dikaitkan dengan Buddha Gautama, umumnya dikenal sebagai Buddha ("yang terbangun"). Menurut tradisi Buddha, Buddha hidup dan mengajar di bagian timur anak benua India, Nepal saat ini sekitar abad ke 6 dan 4 SM. [Catatan 1] Ia diakui oleh umat Buddha sebagai guru yang terbangun atau tercerahkan yang berbagi pengalamannya dengan Buddha. wawasan untuk membantu makhluk hidup mengakhiri penderitaan mereka melalui penghapusan ketidaktahuan dan keinginan. Umat Buddha percaya bahwa ini dicapai melalui pemahaman dan persepsi langsung tentang kemunculan bergantungan dan Empat Kebenaran Mulia.', 'agama nontheistik')
find_string_approx('a Nikaya, yang diakui oleh sebagian besar cendekiawan sebagai teks awal (lih. Baptisan bayi). Buddhisme Tibet terkadang menambahkan perlindungan keempat, dalam lama. Di Mahayana, orang yang memilih jalur bodhisattva membuat sumpah atau janji, dianggap sebagai ekspresi tertinggi dari welas asih. Di Mahayana juga, Tiga Permata dian', 'Tibet')
find_string_approx('Koloni dan komunitas Yunani di tepi Laut Mediterania dan Laut Hitam, tetapi orang-orang Yunani selalu berpusat di laut Aegean dan Ionia, di mana bahasa Yunani telah digunakan sejak Zaman Perunggu. Hingga awal abad ke-20, orang-orang Yunani beralih ke semenanjung Yunani, pantai barat Asia Kecil, pantai Laut Hitam, Kapadokia di Anatolia tengah, Mesi', 'Koloni dan komunitas Yunani secara historis didirikan di tepi Laut Mediterania dan Laut Hitam')
find_string_approx('ercaya bahwa invasi Dorian menyebabkan runtuhnya peradaban Mycenaean, tetapi kemungkinan serangan utama dilakukan oleh perampok pelaut (orang-orang laut) yang berlayar ke Mediterania timur sekitar tahun 1180 SM. Invasi Dorian diikuti oleh periode migrasi yang terbukti buruk, yang secara tepat disebut Abad Kegelapan Yunani, tetapi pada 800 SM lansekap Yunani Kuno dan Klasik sudah terlihat.', 'Diharapkan serangan utama dilakukan oleh perampok pelaut (masyarakat laut)')

(89, 153)

In [9]:
LEFT_OFFSET = 68
RIGHT_OFFSET = 256

IMPOSSIBLE_LOC = (-2, -2)

def get_answer_loc_new(context, answer_text, answer_loc, logger=logger):
    if answer_text == '':
        return IMPOSSIBLE_LOC
    
    answer_text_total_words = len(answer_text.split())
            
    context_loc = (max(answer_loc[0]-len(answer_text)-LEFT_OFFSET, 0), min(answer_loc[0]+len(answer_text)+RIGHT_OFFSET, len(context)))
    prepared_context = context[context_loc[0]:context_loc[1]].lower()
    prepared_answer_text = answer_text.lower()

    answer_loc_start, answer_loc_end = find_string_approx(prepared_context, prepared_answer_text, logger)
    if answer_loc_start != -1:
        answer_loc_new = (
            answer_loc_start + context_loc[0],
            answer_loc_end + context_loc[0]
        )
        logger.debug(f'Indonesia answer_loc_new: {answer_loc_new}')
    else:
        answer_loc_new = (-1, -1)
        logger.debug(context[context_loc[0]:context_loc[1]])
        logger.debug(f'INDONESIA ANSWER_LOC NOT FOUND: {answer_loc_new}')
    return answer_loc_new

In [63]:
def find_new_answers_and_calc_failures(taken_topic_idx, taken_context_idx, logger=logger):
    qas = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas']
    context = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['context']
    logger.debug(f'{context}\n')

    failure_count = 0
    i = 0
    for question in qas:
        logger.debug(i)
        answers = question['answers']
        if len(answers) == 0:
            logger.debug("Impossible question. Alternating to plausible answers...")
            answers_key = 'plausible_answers'
        else:
            answers_key = 'answers'
        try:
            answer_text = '' if len(question[answers_key]) == 0 else question[answers_key][0]['text']
        except Exception as e:
            print(taken_topic_idx)
            print(taken_context_idx)
            print(i)
            raise e
        answer_loc = -1 if len(question[answers_key]) == 0 else int(question[answers_key][0]['answer_start'])
        answer_loc = (min(answer_loc, len(context) - 1), min(answer_loc + len(answer_text), len(context)))
        logger.debug(f'English answer_loc: {answer_loc}')

        answer_loc_new = get_answer_loc_new(context, answer_text, answer_loc, logger)
    
        # If not found, try the original english answer. Probably GoogleTranslate had translated name entities.
        if answer_loc_new == (-1, -1):
            answer_text = df_squad_original.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas'][i][answers_key][0]['text']
            answer_loc_new = get_answer_loc_new(context, answer_text, answer_loc, logger)

        failure_count += 1 if answer_loc_new == (-1, -1) else 0
            
        indonesian_answer_key = f'indonesian_{answers_key}'
        indonesian_answer = [
            {
                'text': answer_text,
                'answer_start': answer_loc_new[0],
                'answer_end': answer_loc_new[1]
            }
        ]
        df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas'][i][indonesian_answer_key] = indonesian_answer
        
        i += 1
        logger.debug(f'Question: {question["question"]}')
        logger.debug(f'Answer: {question[answers_key]}')
        logger.debug(f'Located in context: {context[answer_loc_new[0]:answer_loc_new[1]]}')
        logger.debug('')
    logger.debug(f'Failure counts:{failure_count}')
    df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['failure_count'] = failure_count

In [64]:
for taken_topic_idx in range(df_squad.shape[0]):
    logger.debug(f'Topic: {taken_topic_idx}')
    for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        logger.debug(f'\t{taken_context_idx}')
        find_new_answers_and_calc_failures(taken_topic_idx, taken_context_idx)

In [66]:
df_squad.iloc[-1]['paragraphs'][-1]['qas'][-1]

{'plausible_answers': [{'text': 'kekuatan kilogram', 'answer_start': 383}],
 'question': 'Kekuatan apa yang merupakan bagian dari sistem SI modern?',
 'id': '5ad28ad0d7d075001a4299cf',
 'answers': [],
 'is_impossible': True,
 'indonesian_plausible_answers': [{'text': 'kilogram-force',
   'answer_start': 402,
   'answer_end': 410}]}

In [71]:
TRAIN_SAVE_PATH = 'Datasets/SQuAD/v2.0/train-v2.0-translated_fixed.json'
DEV_SAVE_PATH = 'Datasets/SQuAD/v2.0/dev-v2.0-translated_fixed.json'

df_squad.to_json(DEV_SAVE_PATH)

# Add failure_percentage column

In [67]:
failure_percentage = [0.0 for _ in range(df_squad.shape[0])]
total_questions = [0 for _ in range(df_squad.shape[0])]

for taken_topic_idx in range(df_squad.shape[0]):
    logger.debug(f'Topic: {taken_topic_idx}')
    ctx_total_question = []
    failure_list = []
    
    for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        logger.debug(f'\tContext_idx: {taken_context_idx}')
        context = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]
        qas_total_question = len(context['qas'])
        ctx_failure_percentage = context['failure_count']/qas_total_question
        logger.debug(f'\tFailures: {context["failure_count"]}/{qas_total_question} = {ctx_failure_percentage * 100:.2f}%')
        ctx_total_question.append(qas_total_question)
        failure_list.append(ctx_failure_percentage)
        
    total_questions[taken_topic_idx] = sum(ctx_total_question)
    failure_percentage[taken_topic_idx] = sum(failure_list)/len(failure_list)
    
df_squad['failure_percentage'] = failure_percentage
df_squad['total_questions'] = total_questions

In [68]:
# pd.set_option('display.max_rows', 50)

In [69]:
# df_analysis = df_squad[df_squad.failure_percentage > 0.12].drop(columns=['paragraphs'])
df_analysis = df_squad.drop(columns=['paragraphs'])
print(df_analysis.failure_percentage.describe())
df_analysis['failed_questions'] = df_analysis['failure_percentage'] * df_analysis['total_questions']
df_analysis = df_analysis.sort_values(by=['failed_questions'], ascending=False)
sorted_index = df_analysis.index
print(df_analysis)
print(f"Dropped: {df_analysis['failed_questions'].sum()}")
print(f"Total: {df_analysis['total_questions'].sum()}")

count    35.000000
mean      0.077461
std       0.035322
min       0.020202
25%       0.053465
50%       0.076836
75%       0.100458
max       0.180528
Name: failure_percentage, dtype: float64
                                            title  failure_percentage  \
34                                        Memaksa            0.180528   
22                            Ketimpangan ekonomi            0.109316   
6                                       Mesin uap            0.115459   
29                            Parlemen Skotlandia            0.138390   
31                                   Imperialisme            0.121785   
30                                      Islamisme            0.103134   
11                                     Ctenophora            0.117512   
1                    Teori kompleksitas komputasi            0.089244   
15                                        Geologi            0.098571   
28                                          Rhine            0.068468   
17  

In [70]:
FILE_LOGGER_NAME = 'file_logger'

# File logger
file_logger = logging.getLogger(FILE_LOGGER_NAME)
file_logger.setLevel(logging.DEBUG)

f_handler = logging.FileHandler('answer_findings_log_dev.txt', 'w', 'utf-8')
f_format = logging.Formatter('%(message)s')
f_handler.setFormatter(f_format)

for handler in file_logger.handlers:
    file_logger.removeHandler(handler)    
file_logger.addHandler(f_handler)

for i in range(len(sorted_index)):
    taken_topic_idx = sorted_index[i]
    file_logger.debug(f'Topic {taken_topic_idx}: {df_squad.iloc[taken_topic_idx]["title"]}')
    file_logger.debug(f'Rank: {i}')
    file_logger.debug(f'Failure rate: {df_analysis["failed_questions"][taken_topic_idx]:.0f}/{df_analysis["total_questions"][taken_topic_idx]} = {df_analysis["failure_percentage"][taken_topic_idx]:.2f}%')
    for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        find_new_answers_and_calc_failures(taken_topic_idx, taken_context_idx, file_logger)
        file_logger.debug('---------- END OF CONTEXT ----------')
    file_logger.debug(f'========== END OF TOPIC {taken_topic_idx} ==========\n')